In [3]:
#test to show the git can both push and pull from Deacon, delete later
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")  # or any other suitable solver, like CPLEX, Gurobi, etc.

   Resolving package versions...
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`


In [4]:
Pkg.add("Distributions")
using Distributions

   Resolving package versions...
   Installed FillArrays ─ v1.8.0
   Installed PDMats ───── v0.11.30
    Updating `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  [31c24e10] + Distributions v0.25.103
    Updating `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`
  [49dc2e85] + Calculus v0.5.1
  [31c24e10] + Distributions v0.25.103
  [fa6b7ba4] + DualNumbers v0.6.8
  [1a297f60] + FillArrays v1.8.0
  [34004b35] + HypergeometricFunctions v0.3.23
  [90014a1f] + PDMats v0.11.30
  [1fd47b50] + QuadGK v2.9.1
  [79098fc4] + Rmath v0.7.1
  [4c63d2b9] + StatsFuns v1.3.0
  [f50d1b31] + Rmath_jll v0.4.0+0
  [4607b0f0] + SuiteSparse
Precompiling project...
  ✓ PDMats
  ✓ FillArrays
  ✓ FillArrays → FillArraysSparseArraysExt
  ✓ FillArrays → FillArraysStatisticsExt
  ✓ FillArrays → FillArraysPDMatsExt
  ✓ Distributions
  ✓ Distributions → DistributionsTestExt
  7 dependencies successfully precompiled in 22 seconds. 227 already precompiled. 1 skipped during auto due to previous errors

In [5]:
sedans = rand(Uniform(0,14), 2, 3)
hybrids = rand(Uniform(0,14), 2, 3)
mini = rand(Uniform(0,14), 2, 3)
vans = rand(Uniform(0,14), 2, 3)
trucks = rand(Uniform(0,14), 2, 4) #creating the age sets for each type of vehicle, dimensions are 2 by # of vehicles
fleet = [sedans, hybrids, mini, vans, trucks]
for i in 1:length(fleet)
    for k in 1:size(fleet[i],2)
        fleet[i][1,k] = round(fleet[i][1,k]) #round all vehicle ages to nearest year
        fleet[i][2,k] = 0 #boolean of whether a vehicle is electric
    end
end
fleetMiles = [4000,4000,4000,4000,12000] #based on US survey, car and van miles are 1/3 because fleet services says they are only checked out roughly 1/3 the time while trucks are mostly permanently assigned, same order as variables are initialized
gasCost = [0.18, 12, 23, 32, 24] #rough dollars of gas per mile driven, same order as above
elecCost = [0,0,0,0,0] #rough dollars of electricity per mile driven, same order
newGas = [0,0,0,0,0] #cost of buying a new gas vehicle of type n, same order
newElec = [0,0,0,0,0] #cost of buying a new electric vehicle of type n, same order


5-element Vector{Int64}:
 0
 0
 0
 0
 0

In [6]:
#yearly fleet operation cost
totalYearlyCost = 0
for i in 1:length(fleet)
    for k in 1:size(fleet[i],2)
        if fleet[i][2,k] == 0 #check for electric
            totalYearlyCost += fleetMiles[i]*gasCost[i]
        else 
            totalYearlyCost += fleetMiles[i]*elecCost[i] #could look to add a year modifyer to decrease electricity costs
        end
    end
end


In [ ]:
#function to replace vehicles, needs to be able to modify fleet array somehow

In [7]:
using JuMP
using GLPK  # or your preferred solver

model = Model(GLPK.Optimizer)

# Decision variables
@variable(model, )

# Objective function (minimize the total cost)
@objective(model, Min, )

# Constraints (e.g., only one replacement per year)


LoadError: LoadError: At In[7]:10: `@objective(model, Min)`: needs three arguments: model, objective sense (Max or Min) and expression.
in expression starting at In[7]:10